# Домашнее задание к лекции "Регулярные выражения"
Для зачета необходимо выполнить 4 задания.  
Для зачета с отличием – все.

In [1]:
import re

## Задание 1

Напишите функцию, которая принимает на вход строку и проверяет является ли она валидным транспортным номером. Если да, то функция должна возвращать отдельно номер и регион.

In [14]:
def iscarnumvalid(carnum):
    #validcarnum = 'а123см45'
    valid_chars = '[АВЕКМНОРСТУХ]'
    mask = ''+valid_chars+'\d\d\d'+valid_chars+'{2}\d\d[\d]?' #символ + три цифры + два символа + две цифры + может быть цифра
    if re.fullmatch(mask, carnum, re.IGNORECASE):
        return tuple(filter(None, re.split('(.{6})+',carnum, 0, re.IGNORECASE))) #сложная маска не нужна, мы уже проверили валидность
    else:
        return None
        
iscarnumvalid('а123см456')

('а123см', '456')

## Задание 2

Напишите функцию, которая проверяет e-mail адрес на валидность по стандарту RFC822: http://www.w3.org/Protocols/rfc822/. Для этого постарайтесь написать (в крайнем случае найти) самый полный шаблон для проверки e-mail адресов через регулярное выражение. Разберитесь и напишите примеры, какие ваш шаблон может не охватить.

In [20]:

def isemailvalid(mailadr):
    mask = r'^(?=.{0,64}\@)((\"?)[\w\-\_\+]+\2)\@(?=.{0,255}$)([\w]+([\.(?!\.)]\w+)*$)'
# такая страшная маска вышла:
#     ^ чтобы валидность была с начала строки
#     (?=.{0,64}\@) - подобная группа была подсмотрена в гугле на ограничение длины строки и адаптирована под поиск с@баки
#     (\"?) по стандарту вроде как можно начать строку с кавычек, если в строке есть странные символы (которые нынче банятся общественными почтовиками)
#          эта же группа через \2 в конце ищет то же что вернет ее первое упоминание
#     [\w\-\_\+] символы, какие хочется, можно расширить список
#     \@ собака
#     (?=.{0,255}$) ограничение имени домена в 255 символов до конца строки
#     ([\w]+([\.(?!\.)]\w+)*$) это далось сложно, в начале домена должен быть символ, а дальше сколько угодно групп вида точка+символы, при этом лукэхед на точку после точки
#          а чтобы это все не развалилось в конце символ поиска строки
# было сложно
    print('+' if re.fullmatch(mask, mailadr) else '-')
#     какие-нибудь имэйлы наверное не сможет, стандарт оч странно писан, и явно не имэйлы его целью были

mailadr = '"hop-hey"@la.la.ley'
isemailvalid(mailadr)

+


## Задание 3
При помощи регулярных выражений напишите функцию, которая будет возвращать акроним переданного в нее текста.

In [21]:
def acro(word):
    mask = r'^\w|(?<=[\W])\w'
    mask = r'(?<=\b)\w'  #про \b в лекции не сказано, а полезно
    print(re.findall(mask,word))
    return ''.join(re.findall(mask,word)).upper()

acro('При помощи регулярных выражений напишите функцию')

['П', 'п', 'р', 'в', 'н', 'ф']


'ППРВНФ'

## Задание 4
Напишите функцию, которая будет удалять все последовательные повторы слов из заданной строки.

In [22]:
def removerepeats(string):
    mask = r'\b([a-zA-Zа-яА-Я]+)(?=\W+\1)'
    #\b([a-zA-Z]+) начало слова и группа букв
    #(?=\W+\1) лукэхэд незначащие символы + группа №1 букв
    return re.sub(mask, '', string)
    
    
string ='удалять удалять удалять все последовательные последовательные последовательные повторы но не пробелы пробелы пробелы'
removerepeats(string)

'  удалять все   последовательные повторы но не   пробелы'

## Задание 5
Напишите функцию, которая будет проверять номер сотового телефона на валидность, если он валиден, то переводить его в формат:   
+7(xxx)-xxx-xx-xx  
Постарайтесь предусмотреть как можно больше адекватных форматов изначального ввода номера.
Примеры правильных и неправильных форматов:
- +1 555 555-55-55;
- 5555555555;
- 555555555;
- 8(555)555-55-55;
- +7 555 555 55 55;
- 7(555) 555-55-55;
- +7 555+555+55+55.

In [23]:
def iscellphonenumvalid(cellnum):
    mask = r'^([\+]?[78]?)?[\s\(]?\d{3}[\s\)]?([\-\+\s]?\d){7}$'
    #^ начало строки
    #([\+]?[78]?)? возможна группа +7/+8/7/8
    #?[\s\(]?\d{3}[\s\)]? три цифры оператора, которые могут быть отделены пробелами или скобками
    #([\-\+\s]?\d){7} семь групп по одной цифре отделенной разделителем
    #$ конец строки
    return True if re.fullmatch(mask, cellnum) else False

def convertcellnum(cellnum):
    num = re.sub(r'[\s\-\+\(\)]','',cellnum)
    firstdpos = 0 if len(num)==10 else 1
    converted = '+7('+num[firstdpos:firstdpos+3]+')-'+num[-7:-4]+'-'+num[-4:-2]+'-'+num[-2:]
    return converted

phnlst = ['+1 555 555-55-55','5555555555','555555555','8(555)555-55-55','7(555) 555-55-55','+7 555+555+55+55','(123)456 78-90']
for phnn in phnlst:
    print(f'{phnn}  {"valid: "+convertcellnum(phnn) if iscellphonenumvalid(phnn) else "non valid"}')

# mask =r'^([\+]?[78]?)?[\s\(]?(\d{3})[\s\)]?(([\-\s]?\d){7})$'          
# re.split(mask, phnlst[1])

+1 555 555-55-55  non valid
5555555555  valid: +7(555)-555-55-55
555555555  non valid
8(555)555-55-55  valid: +7(555)-555-55-55
7(555) 555-55-55  valid: +7(555)-555-55-55
+7 555+555+55+55  valid: +7(555)-555-55-55
(123)456 78-90  valid: +7(123)-456-78-90
